In [5]:
import mlflow
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from IPython.display import clear_output
import time
# from ray_on_azureml.ray_on_aml import getRay
import sys
sys.path.append("../") # go to parent dir
import importlib


In [6]:
from src.ray_on_azureml.ray_on_aml import Ray_On_AML
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="worker-cpu-v3")
# _, ray = ray_on_aml.getRay()
# ray.cluster_resources()

ModuleNotFoundError: No module named 'ray'

In [3]:
ray_on_aml.shutdown()
# import ray
# ray.shutdown()
# ray.init()

NameError: name 'ray_on_aml' is not defined

In [6]:
# import ray
# ray.init()
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
# ray.shutdown()
# client = ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True  )
# with client:
# Start Ray.
# Tip: If connecting to an existing cluster, use ray.init(address="auto").
# ray.init(address ='ray://10.0.0.14:10001',ignore_reinit_error=True)
d_arr = da.from_array(np.random.randint(0, 1000, size=(256, 256)))

# The Dask scheduler submits the underlying task graph to Ray.
d_arr.mean().compute(scheduler=ray_dask_get)

# Set the scheduler to ray_dask_get in your config so you don't have to
# specify it on each compute call.
dask.config.set(scheduler=ray_dask_get)

df = dd.from_pandas(
    pd.DataFrame(
        np.random.randint(0, 10000, size=(1024, 2)), columns=["age", "grade"]),
    npartitions=2)
df.groupby(["age"]).mean().compute()

# ray.shutdown()

,grade
age,
14,2054.0
29,4682.0
72,4525.0
94,1078.0
118,9947.0
...,...
9940,2479.0
9955,8684.0
9961,128.0


In [1]:
pip install fsspec==2021.10.1

  Using cached fsspec-2021.10.1-py3-none-any.whl (125 kB)
ERROR: datasets 1.8.0 has requirement pyarrow<4.0.0,>=1.0.0, but you'll have pyarrow 5.0.0 which is incompatible.
ERROR: datasets 1.8.0 has requirement tqdm<4.50.0,>=4.27, but you'll have tqdm 4.61.2 which is incompatible.
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.7.0
    Uninstalling fsspec-2021.7.0:
      Successfully uninstalled fsspec-2021.7.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade pyarrow 

Requirement already up-to-date: pyarrow in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (6.0.1)


In [9]:
import dask.dataframe as dd

storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)
ddf.count()

Dask Series Structure:
npartitions=1
doLocationId    int64
vendorID          ...
dtype: int64
Dask Name: dataframe-count-agg, 17 tasks

In [6]:
import dask
dask.__version__
# '2021.09.0'
# '2.30.0'


'2021.09.0'

In [3]:
import pyarrow
pyarrow.__version__

'4.0.1'

In [2]:
pip install --upgrade cchardet, pip install -U pyarrow

     |████████████████████████████████| 265 kB 11.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install -U pyarrow

  Using cached pyarrow-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25.6 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 3.0.0
    Uninstalling pyarrow-3.0.0:
      Successfully uninstalled pyarrow-3.0.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

azureml-dataset-runtime 1.34.0 requires pyarrow<4.0.0,>=0.17.0, but you'll have pyarrow 6.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyarrow==4.0.1

  Using cached pyarrow-4.0.1-cp38-cp38-manylinux2014_x86_64.whl (21.9 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

azureml-dataset-runtime 1.34.0 requires pyarrow<4.0.0,>=0.17.0, but you'll have pyarrow 4.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.uniform(0.01, 0.09)
}

# Uncomment this to enable distributed execution
# ray.shutdown()
# ray.init(address="auto",ignore_reinit_error=True)
# ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True,ignore_reinit_error=True )
# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(train_mnist, config=search_space)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /home/azureuser/data/MNIST/raw/train-images-idx3-ubyte.gz to /home/azureuser/data/MNIST/raw

Extracting /home/azureuser/data/MNIST/raw/train-labels-idx1-ubyte.gz to /home/azureuser/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /home/azureuser/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/azureuser/data/MNIST/raw

Extracting /home/azureuser/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/azureuser/data/MNIST/raw

Processing...


  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Trial name,status,loc,lr,momentum
train_mnist_9507e_00000,PENDING,,3.72805e-10,0.0233414


(pid=5668) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
(pid=5668)   return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Result for train_mnist_9507e_00000:
  date: 2021-12-10_02-48-00
  done: false
  experiment_id: f0d5d864bb194a29a0b5fe918e61a97d
  hostname: janguy2
  iterations_since_restore: 1
  mean_accuracy: 0.053125
  node_ip: 10.0.0.5
  pid: 5668
  time_since_restore: 1.2203056812286377
  time_this_iter_s: 1.2203056812286377
  time_total_s: 1.2203056812286377
  timestamp: 1639104480
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9507e_00000
  
Result for train_mnist_9507e_00000:
  date: 2021-12-10_02-48-01
  done: true
  experiment_id: f0d5d864bb194a29a0b5fe918e61a97d
  experiment_tag: 0_lr=3.7281e-10,momentum=0.023341
  hostname: janguy2
  iterations_since_restore: 10
  mean_accuracy: 0.053125
  node_ip: 10.0.0.5
  pid: 5668
  time_since_restore: 2.5444483757019043
  time_this_iter_s: 0.14707326889038086
  time_total_s: 2.5444483757019043
  timestamp: 1639104481
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 9507e_00000
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_9507e_00000,TERMINATED,10.0.0.5:5668,3.72805e-10,0.0233414,0.053125,10,2.54445


2021-12-10 02:48:01,935	INFO tune.py:630 -- Total run time: 4.09 seconds (3.52 seconds for the tuning loop).


In [11]:
 import sklearn.datasets
 import sklearn.metrics
 from sklearn.model_selection import train_test_split
 import xgboost as xgb

 from ray import tune


 def train_breast_cancer(config):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     tune.report(mean_accuracy=accuracy, done=True)


 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3]),
     "subsample": tune.uniform(0.5, 1.0),
     "eta": tune.loguniform(1e-4, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 1},
     config=config,
     num_samples=10)


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_98d5b_00000,PENDING,,0.000215207,1,2,0.991314
train_breast_cancer_98d5b_00001,PENDING,,0.00177882,4,2,0.860054
train_breast_cancer_98d5b_00002,PENDING,,0.0237837,6,3,0.577541
train_breast_cancer_98d5b_00003,PENDING,,0.000448295,7,3,0.669501
train_breast_cancer_98d5b_00004,PENDING,,0.000131317,6,2,0.979965
train_breast_cancer_98d5b_00005,PENDING,,0.00842119,2,3,0.691144
train_breast_cancer_98d5b_00006,PENDING,,0.00727616,5,2,0.650641
train_breast_cancer_98d5b_00007,PENDING,,0.00647503,6,2,0.717976
train_breast_cancer_98d5b_00008,PENDING,,0.000159053,8,1,0.519957
train_breast_cancer_98d5b_00009,PENDING,,0.000163399,7,3,0.783772


Result for train_breast_cancer_98d5b_00002:
  date: 2021-12-10_02-48-05
  done: true
  experiment_id: 20cd17427a7441799255e975f9ee7215
  hostname: janguy2
  iterations_since_restore: 1
  mean_accuracy: 0.902098
  node_ip: 10.0.0.5
  pid: 5669
  time_since_restore: 0.03443479537963867
  time_this_iter_s: 0.03443479537963867
  time_total_s: 0.03443479537963867
  timestamp: 1639104485
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 98d5b_00002
  
Result for train_breast_cancer_98d5b_00000:
  date: 2021-12-10_02-48-05
  done: true
  experiment_id: 1973cc8ffc57478eb86103dc93e2a5c5
  hostname: janguy2
  iterations_since_restore: 1
  mean_accuracy: 0.93007
  node_ip: 10.0.0.5
  pid: 5724
  time_since_restore: 0.01591324806213379
  time_this_iter_s: 0.01591324806213379
  time_total_s: 0.01591324806213379
  timestamp: 1639104485
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 98d5b_00000
  
Result for train_breast_cancer_98d5b_00001:
  date: 2021-12-10_02-48-05

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_98d5b_00000,TERMINATED,10.0.0.5:5724,0.000215207,1,2,0.991314,0.93007,1,0.0159132
train_breast_cancer_98d5b_00001,TERMINATED,10.0.0.5:5723,0.00177882,4,2,0.860054,0.93007,1,0.0214803
train_breast_cancer_98d5b_00002,TERMINATED,10.0.0.5:5669,0.0237837,6,3,0.577541,0.902098,1,0.0344348
train_breast_cancer_98d5b_00003,TERMINATED,10.0.0.5:10730,0.000448295,7,3,0.669501,0.923077,1,0.0189586
train_breast_cancer_98d5b_00004,TERMINATED,10.0.0.5:10735,0.000131317,6,2,0.979965,0.93007,1,0.0219936
train_breast_cancer_98d5b_00005,TERMINATED,10.0.0.5:10710,0.00842119,2,3,0.691144,0.888112,1,0.0169973
train_breast_cancer_98d5b_00006,TERMINATED,10.0.0.5:10711,0.00727616,5,2,0.650641,0.937063,1,0.0242865
train_breast_cancer_98d5b_00007,TERMINATED,10.0.0.5:10708,0.00647503,6,2,0.717976,0.93007,1,0.0215454
train_breast_cancer_98d5b_00008,TERMINATED,10.0.0.5:10709,0.000159053,8,1,0.519957,0.979021,1,0.0204871
train_breast_cancer_98d5b_00009,TERMINATED,10.0.0.5:10707,0.000163399,7,3,0.783772,0.895105,1,0.0192904


2021-12-10 02:48:06,189	INFO tune.py:630 -- Total run time: 1.95 seconds (1.81 seconds for the tuning loop).


In [3]:
#dask

# import ray
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import dask
import glob
import joblib

import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime

from azureml.core import Workspace, Dataset, Model
data_url = 'https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather'



In [5]:
from adlfs import AzureBlobFileSystem,AzureDatalakeFileSystem
sas_token='sp=rle&st=2021-11-27T19:26:49Z&se=2022-04-22T02:26:49Z&spr=https&sv=2020-08-04&sr=c&sig=gpYJ3LofkkjkeWkrMDTY1Ge6NsOCghbUR9CQs65HCvU%3D'
account_key='ak7nSgU/iRc5XCvXjaM2UwX18ybS9WO1BsE4Vm42XLmiZvBm1kCyZQD3USeCCOiwusWkDaq0jcRF1JpPHiAMhw=='
account_name="adlsgen7"
abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")
abfs2 = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")


In [12]:
import dask.dataframe as dd

storage_options={'account_name': account_name, 'account_key': account_key}

ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year2008', storage_options=storage_options)


ValueError: Directory name did not appear to be a partition: month

In [6]:
# https://adlsgen7.blob.core.windows.net/mltraining/ISDWeatherDelta/year2008/month/part-00009-21953d01-b5b8-4fa7-8bd2-96b025a24bc3.c000.snappy.parquet
# https://adlsgen7.blob.core.windows.net/mltraining/ISDWeatherDelta/year2008/month10/part-00001-1b0e76df-8a22-4221-9b5a-8b7936aa13ea.c000.snappy.parquet
data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs2)
data2 = ray.data.read_parquet("az://mltraining/ISDWeatherDelta/year2008", filesystem=abfs)

Metadata Fetch Progress: 100%|██████████| 8/8 [00:00<00:00, 14.77it/s]


In [17]:
data2.count()

97581959

In [14]:
ddf.count().compute()

usaf                       97581959
wban                       97581959
datetime                   97581959
latitude                   97581959
longitude                  97581959
elevation                  97581959
windAngle                  79483952
windSpeed                  82288222
temperature                95119398
seaLvlPressure             35943606
cloudCoverage              57138202
presentWeatherIndicator     9538179
pastWeatherIndicator        4114610
precipTime                 19003392
precipDepth                19003392
snowDepth                    353685
stationName                97319338
countryOrRegion            97267449
p_k                        97581959
day                        97581959
version                    97581959
dtype: int64

In [13]:
dask.config.set(scheduler=ray_dask_get)


In [15]:
data.count()

98904376

In [1]:
import ray
import raydp
import os
ray.shutdown()
ray.init()
os.environ["PYSPARK_PYTHON"]="/anaconda/envs/azureml_py38/bin/python3"

# ray.init(address ='ray://10.0.0.11:6379')
spark = raydp.init_spark(
  app_name = "example",
  num_executors = 2,
  executor_cores = 1,
  executor_memory = "1gb"
)

# data =spark.read.format("csv").option("header", True).load("wasbs://ojsales-simulatedcontainer@azureopendatastorage.blob.core.windows.net/oj_sales_data/Store10*.csv")


# # normal data processesing with Spark
# df = spark.createDataFrame([('look',), ('spark',), ('tutorial',), ('spark',), ('look', ), ('python', )], ['word'])
# df.show()
# word_count = df.groupBy('word').count()
# word_count.show()
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b

multiply = pandas_udf(multiply_func, returnType=LongType())

# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))
# 0    1
# 1    4
# 2    9
# dtype: int64

# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()
# +-------------------+
# |multiply_func(x, x)|
# +-------------------+
# |                  1|
# |                  4|
# |                  9|
# +-------------------+


# stop the spark cluster
raydp.stop_spark()


2021-12-06 19:48:42,187	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266


2021-12-06 19:48:45,043 WARN Utils [Thread-1]: Your hostname, nc6v0 resolves to a loopback address: 127.0.0.1; using 10.0.0.11 instead (on interface eth0)
2021-12-06 19:48:45,046 WARN Utils [Thread-1]: Set SPARK_LOCAL_IP if you need to bind to another address
2021-12-06 19:48:45,240 WARN NativeCodeLoader [Thread-1]: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2021-12-06 19:48:45,321 INFO SecurityManager [Thread-1]: Changing view acls to: azureuser
2021-12-06 19:48:45,321 INFO SecurityManager [Thread-1]: Changing modify acls to: azureuser
2021-12-06 19:48:45,322 INFO SecurityManager [Thread-1]: Changing view acls groups to: 
2021-12-06 19:48:45,322 INFO SecurityManager [Thread-1]: Changing modify acls groups to: 
2021-12-06 19:48:45,323 INFO SecurityManager [Thread-1]: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(azureuser); groups with view permissions: Set(); users  with modify permissions: Set(azureuser); groups with modify permissions: Set()
2021-12-06 19:48:45,591 INFO Utils [Thread-1]: Successfully started service 'RAY_RPC_ENV' on port 44621.


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/jars/ray_dist.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2021-12-06 19:48:48,798 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registering app example
2021-12-06 19:48:48,802 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registered app example with ID app-20211206194848-0000
0    1
1    4
2    9
dtype: int64


(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
(raylet) WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
(raylet) WARNING: All illegal access operations will be denied in a future release
(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsa

+-------------------+
|multiply_func(x, x)|
+-------------------+
|                  1|
|                  4|
|                  9|
+-------------------+

2021-12-06 19:48:59,086 INFO RayAppMaster [Thread-1]: Stopping RayAppMaster


In [ ]:
raydp.stop_spark()
